In [2]:
print(1)

1


In [3]:
import psutil
import time
import threading
from functools import wraps


def monitor_memory_decorator(interval=1):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Start monitoring in a separate thread
            running = True

            def monitor_memory():
                process = psutil.Process()  # Get the current Python process
                try:
                    while running:
                        mem_info = process.memory_info()
                        print(
                            f"Memory Usage: RSS = {mem_info.rss / (1024 ** 2):.2f} MB, "
                            + f"VMS = {mem_info.vms / (1024 ** 2):.2f} MB"
                        )
                        time.sleep(interval)
                finally:
                    print("Memory monitoring stopped.")

            # Thread setup
            monitor_thread = threading.Thread(target=monitor_memory)
            monitor_thread.start()

            # Execute the function
            result = func(*args, **kwargs)
            running = False

            return result

        return wrapper

    return decorator


In [4]:
import subprocess
import os

@monitor_memory_decorator(interval=1)
def split_audio_in_place_ffmpeg(audio_path, segment_duration):
    directory, filename = os.path.split(audio_path)
    basename, _ = os.path.splitext(filename)
    segment_dir = os.path.join(directory, f"{basename}_segments")
    os.makedirs(segment_dir, exist_ok=True)

    # Construct the ffmpeg command to split the audio
    segment_path = os.path.join(segment_dir, f"{basename}_segment%03d.mp3")
    command = [
        'ffmpeg',
        '-i', audio_path,           # Input file
        '-f', 'segment',            # Format to use for segmenting
        '-segment_time', str(segment_duration),  # Duration of each segment in seconds
        '-c', 'copy',               # Use the same codec
        segment_path                # Output path pattern
    ]

    # Execute the command
    subprocess.run(command, check=True)

    # Optionally, remove the original file if required
    # os.remove(audio_path)

In [5]:

# Example usage
audio_path = 'sample_data/2024-05-22 -140339_1920x1170.mp3'
segment_duration = 300  # 1 minute segments
split_audio_in_place_ffmpeg(audio_path, segment_duration)


Memory Usage: RSS = 31.86 MB, VMS = 402265.25 MB


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --en